In [19]:
# implement semantic similarity evaluation between two strings using edit distance                          
import Levenshtein

# Input strings
string1 = "kitten"
string2 = "sitting"

# Calculate Levenshtein distance
distance = Levenshtein.distance(string1, string2)

# Calculate semantic similarity
max_len = max(len(string1), len(string2))
similarity = 1 - (distance / max_len) if max_len > 0 else 1.0

# Output results
print(f"Levenshtein Distance: {distance}")
print(f"Semantic Similarity: {similarity:.2f}")


Levenshtein Distance: 3
Semantic Similarity: 0.57


In [20]:
import sacrebleu
from rouge import Rouge

# Input strings
reference = "The cat is sitting on the mat."
candidate = "The feline is sitting on the carpet."

# SacreBLEU Calculation
bleu = sacrebleu.sentence_bleu(candidate, [reference]).score

# ROUGE Score Calculation
rouge = Rouge()
reference_sent = reference + " ."  # Add period to make it a valid sentence
candidate_sent = candidate + " ."
scores = rouge.get_scores(candidate_sent, reference_sent, avg=True)

# Output Results
print(f"BLEU Score (sacrebleu): {bleu:.4f}")
print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")  # F1 Score for ROUGE-1
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")  # F1 Score for ROUGE-2
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")  # F1 Score for ROUGE-L


BLEU Score (sacrebleu): 38.2603
ROUGE-1: 0.7500
ROUGE-2: 0.4286
ROUGE-L: 0.7500
